# Vector Space Models

This chapter discusses computational models to represent text in vector spaces.

* [Preparation](#Preparation)
* [Bag-of-Words](#Bag-of-Words)
* [Frequency Counts](#Frequency-Counts)
* [Exercise 1](#Exercise-1)
* [TF-IDF](#TF-IDF)
* [Similarity Metrics](#Similarity-Metrics)

## References

* [Vector Space Model](https://en.wikipedia.org/wiki/Vector_space_model)
* [Bag-of-Words Model](https://en.wikipedia.org/wiki/Bag-of-words_model)
* [TF-IDF](https://en.wikipedia.org/wiki/Tf–idf)
* [Euclidean Distance](https://en.wikipedia.org/wiki/Euclidean_distance)
* [Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity)

## Preparation

Download [`aesopfables.json`](https://github.com/emory-courses/computational-linguistics/blob/master/docs/res/aesopfables.json) and read the JSON file.

In [35]:
!poetry add requests
import requests

def download(remote_addr: str, local_addr: str):
    r = requests.get(remote_addr)
    fin = open(local_addr, 'wb')
    fin.write(r.content)

The following packages are already present in the pyproject.toml and will be skipped:

  • requests

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


* [`requests`](https://requests.readthedocs.io/en/master/user/quickstart/)

In [36]:
aesop_link = 'https://raw.githubusercontent.com/emory-courses/computational-linguistics/master/res/vsm/aesopfables.json'
aesop_file = '../res/vsm/aesopfables.json'
download(aesop_link, aesop_file)

* Make sure which directory `aesopfables.json` is downloaded.

In [37]:
import json
fables = json.load(open(aesop_file))

print(len(fables))
for fable in fables[:10]: print(fable['title'])

357
Androcles
The Ant and the Chrysalis
The Ant and the Dove
The Ants and the Grasshopper
The Apes and the Two Travelers
The Ass and His Driver
The Ass and His Masters
The Ass and His Purchaser
The Ass and His Shadow
The Ass and the Charger


## Bag-of-Words

Let there be a giant bag that can hold all unique words in the world.
Then, each token in a text such as "*Jinho Choi is a professor at Emory University .*" can be inserted to the bag as follows:

<img src="res/bow.jpg" width=600 align="left"/>

In [38]:
This bag can be represented by a vector of which every dimension stands for a unique token in the world.
All dimensions are initialized to `0`, except for the ones representing tokens in the input text, which are assigned with `1`:

<img src="res/vsm.jpg" width=600 align="left"/>

SyntaxError: invalid syntax (3416902580.py, line 1)

* What is the total dimension of this vector?
8
* Does this vector correctly represent the original text (anything missing)?
Order?

A bag-of-words can be implemented by a dictionary (representing a sparse vector), where the key is a term in the text and its value is always `1`.
The value of every other term that does not appear in the document is assumed to be `0`.

In [ ]:
v = {'Jinho': 1, 'Choi': 1, 'is': 1, 'a': 1, 'professor': 1, 'at': 1, 'Emory': 1, 'University': 1, '.': 1}

## Frequency Counts

Consider the following two documents:

```
D1: John bought a car . The car was fancy .
D2: Mary liked the car .  John gave it to Mary .
```

A **term frequency** (`tf`) is the number of occurrences of a specific term in a specific document:

```
tf(John, D1) = 1
tf(John, D2) = 1
tf(Mary, D2) = 2
```

A **document frequency** (`df`) is the number of documents containing a specific term:

```
df(John) = 2
df(John) = 2
```

Let us define the function `term_frequencies()` that takes `fables` above and returns a dictionary where each key-value pair represents the source and term frequencies of the corresponding document:

In [ ]:
from collections import Counter
from typing import Dict

def term_frequencies(fables) -> Dict[str, Counter]:
    def key(t): return t[t.rfind('&')+1:]
    return {key(fable['source']): Counter(fable['tokens'].split()) for fable in fables}

* [`collections.Counter`](https://docs.python.org/3/library/collections.html#collections.Counter)

In [ ]:
tfs = term_frequencies(fables)
print(tfs['Androcles'])

## Exercise 1

Let us define the function `document_frequencies()` that takes `fables` and returns a dictionary where each key-value pair represents a term and its document frequency:

```python
def document_frequencies(fables) -> Dict[str, int]:
    # To be filled
    return dict()
```

In [39]:
def document_frequencies(fables) -> Dict[str, int]:
    tf = term_frequencies(fables)

    doc_freqs = {}
    for doc in tf:
        for term in dict(tf[doc]):
            if term in doc_freqs:
                doc_freqs[term] += 1
            else:
                doc_freqs[term] = 1
    return doc_freqs


dfs = document_frequencies(fables)
print(dfs['Lion'], dfs['lion'])
for term, count in sorted(dfs.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(term, count)

43 5
. 355
, 354
the 349
and 343
to 336
a 334
of 312
" 280
in 274
his 252


* [sorted()](https://docs.python.org/3/library/functions.html?highlight=sorted#sorted)
* [lambda](https://docs.python.org/3/reference/expressions.html#lambda)

## TF-IDF

What are important terms in a document?

* High term frequency
* low document frequency

The $\mathrm{tf}\cdot\mathrm{idf}_{t,d}$ (Term Frequency - Inverse Document Frequency) of a specific term $t$ in a specific document $d \in D$ is measured as follows:

$$
\mathrm{tf}\cdot\mathrm{idf}_{t,d} = \mathrm{tf}_{t,d} \times \log\frac{|D|}{\mathrm{df}_t}
$$

Several variations of $\mathrm{tf}_{t,d}$ have also been proposed using sublinear or normalization:

$$
\begin{align}
\mathrm{wf}_{t,d} &=& \left\{
\begin{array}{cl}
 1 + \log\mathrm{tf}_{t,d} & \mbox{if $\textrm{tf}_{t,d} > 0$}\\
 0 & \mbox{otherwise}
\end{array}
\right. \\
\mathrm{ntf}_{t,d} &=& \alpha + (1-\alpha)\frac{\mathrm{tf}_{t,d}}{\mathrm{tf}_{\mathrm{argmax}({\mathrm{tf}_{\forall t, d}}),d}}
\end{align}
$$


Let us define the function `tf_idfs()` that takes `fables` and returns a dictionary where the key is a (term, document ID) pair, and the value is its TF-IDF score:

In [40]:
from typing import Tuple
import math

def tf_idfs(fables) -> Dict[str, Dict[str, int]]:
    tfs = term_frequencies(fables)
    dfs = document_frequencies(fables)
    out = dict()
    D = len(tfs)

    for dkey, term_counts in tfs.items():
        out[dkey] = {t: tf * math.log(D / dfs[t]) for t, tf in term_counts.items()}

    return out

In [41]:
tfidfs = tf_idfs(fables)
print(tfidfs['Androcles']['Lion'])

19.02357553642621


In [42]:
for t, score in sorted(tfs['Androcles'].items(), key=lambda x: x[1], reverse=True)[:20]:
    print(t, score)

NameError: name 'tfs' is not defined

In [43]:
for t, score in sorted(tfidfs['Androcles'].items(), key=lambda x: x[1], reverse=True)[:20]:
    print(t, score)

Androcles 46.99944584681624
Lion 19.02357553642621
slave 13.465909109196419
Emperor 11.74986146170406
paw 9.55263688436784
thorn 9.55263688436784
loose 8.97727273946428
dog 7.858041163593434
turned 6.471746802473543
forest 5.969117945911732
wandering 5.87493073085203
moaning 5.87493073085203
flee 5.87493073085203
bleeding 5.87493073085203
sentenced 5.87493073085203
arena 5.87493073085203
bounding 5.87493073085203
recognised 5.87493073085203
fawned 5.87493073085203
licked 5.87493073085203


## Similarity Metrics

Given two vectors, $X_i = (x_{i1}, \ldots, x_{in})$ and $X_j = (x_{j1}, \ldots, x_{jn})$, the **Euclidean distance** between $X_i$ and $X_j$ measures the magnitude between them:

$$
\mathrm{Euclidean}(X_i, X_j) = \lVert X_i - X_j \rVert = \sqrt{\sum_{k=1}^n (x_{ik} - x_{jk})^2}
$$

On the other hand, the **Cosine similarity** measures the angle difference between them:


$$
\mathrm{Cosine}(X_i, X_j) = \frac{X_i\cdot X_j}{\lVert X_i\rVert\lVert X_j\rVert} = \frac{\sum_{\forall k}(x_{ik} \cdot x_{jk})}{\sqrt{\sum_{\forall k}(x_{ik})^2} \cdot \sqrt{\sum_{\forall k}(x_{jk})^2}}
$$

<img src="res/vector_similarities.jpg" width=350/>

Let us define the function `euclidean(x1, x2)` that takes two sparse vectors, `x1` and `x2`, and returns their Euclidean distance:

In [44]:
def euclidean(x1: Dict[str, float], x2: Dict[str, float]) -> float:
    t = sum(((s1 - x2.get(term, 0)) ** 2 for term, s1 in x1.items()))
    t += sum((s2 ** 2 for term, s2 in x2.items() if term not in x1))
    return math.sqrt(t)

In [45]:
tfidfs = tf_idfs(fables)

x1 = tfidfs['Androcles']
x2 = tfidfs['TheAntandtheChrysalis']
x3 = tfidfs['TheAntsandtheGrasshopper']
print(euclidean(x1, x2))
print(euclidean(x2, x3))

89.06699680755206
61.724694998008005


Download another JSON file containing alternative Aesop's fables:

In [46]:
link = 'https://raw.githubusercontent.com/emory-courses/computational-linguistics/master/res/vsm/aesopfables-alt.json'
file = '../res/vsm/aesopfables-alt.json'
download(link, file)

In [47]:
fables_alt = json.load(open(file))
for f in fables_alt: print(f['title'])

The Ant and the Grasshopper
The Ass and his Purchaser
The Ass and the Lapdog
The Ass in the Lion Skin
The Belly and the Members
The Buffoon and the Countryman
The Crow and the Pitcher
The Dog in the Manger
The Dog and the Shadow
The Eagle and the Arrow
The Fox and the Crow
The Fox and the Goat
The Fox and the Grapes
The Fox and the Lion
The Fox and the Mask
The Hare and the Tortoise
The Hares and the Frogs
The Horse and the Ass
The Lion and the Mouse
The Lion in Love
The Man and the Satyr
Mercury and the Woodman
The Milkmaid and Her Pail
The Old Man and Death
The Old Woman and the Wine-Jar
The One-Eyed Doe
The Peacock and Juno
The Rose and the Amaranth
The Serpent and the Eagle
The Shepherd's Boy
The Sick Lion
The Town Mouse and the Country Mouse
The Trumpeter Taken Prisoner
The Two Pots
The Vain Jackdaw
The Wolf and the Crane
The Wolf and the Lamb
The Wolf in Sheep's Clothing


In [48]:
tfidf_alt = tf_idfs(fables_alt)
print(tfidf_alt.keys())

dict_keys(['antgrass2.ram', 'TheAssandhisPurchaser2', 'TheAssandtheLapdog2', 'TheAssintheLionSkin', 'TheBellyandtheMembers2', 'TheBuffoonandtheCountryman', 'TheCrowandthePitcher2', 'TheDogintheManger2', 'TheDogandtheShadow2', 'TheEagleandtheArrow2', 'TheFoxandtheCrow2', 'TheFoxandtheGoat2', 'TheFoxandtheGrapes2', 'TheFoxandtheLion', 'TheFoxandtheMask', 'haretort2.ram', 'harefrog2.ram', 'TheHorseandtheAss2', 'TheLionandtheMouse2', 'TheLioninLove2', 'TheManandtheSatyr2', 'MercuryandtheWoodman', 'milkpail2.ram', 'TheOldManandDeath2', 'TheOldWomanandtheWine-Jar', 'TheOne-EyedDoe', 'ThePeacockandJuno', 'TheRoseandtheAmaranth', 'TheSerpentandtheEagle', 'shepherd2.ram', 'TheSickLion2', 'TheTownMouseandtheCountryMouse', 'TheTrumpeterTakenPrisoner2', 'TheTwoPots2', 'TheVainJackdaw', 'TheWolfandtheCrane2', 'TheWolfandtheLamb2', 'TheWolfinSheepsClothing2'])


## Exercise 2

Write a function `most_similar()` that takes a spare vector representation of a document and find the most similar fable among the ones in [aesopfables.json](https://github.com/emory-courses/computational-linguistics/blob/master/res/vsm/aesopfables.json).

```python
def most_similar(Y: Dict[str, Dict[str, float]], x: Dict[str, float]) -> str:
    # To be filled
    return ''
```

In [70]:
import numpy as np

def magnitude(vec) -> float:
    return np.linalg.norm(vec)

def buildVector(vec1, vec2):   # shamelessly stolen off stackexchange
    counter1 = Counter(vec1)
    counter2= Counter(vec2)
    all_items = set(counter1.keys()).union( set(counter2.keys()) )
    vector1 = np.array([counter1[k] for k in all_items])
    vector2 = np.array([counter2[k] for k in all_items])
    return vector1, vector2


def cos_difference(x,y) -> float:
    return np.inner(x,y) / (magnitude(x) * magnitude(y))

def most_similar(Y: dict[str, dict[str, float]], x: dict[str, float]) -> str:
    most_simliar_so_far = ['Androcles', 0]
    for fable in Y:
        hell,yeah = buildVector(x, Y[fable])
        if cos_difference(hell,yeah) >= most_simliar_so_far[1]:
            most_simliar_so_far[0] = fable
            most_simliar_so_far[1] = cos_difference(hell, yeah)

    return most_simliar_so_far[0]

for k, x in tfidf_alt.items():
    t = most_similar(tfidfs, x)
    print('{} -> {}'.format(k, t))

antgrass2.ram -> TheAntsandtheGrasshopper
TheAssandhisPurchaser2 -> TheAssandHisPurchaser
TheAssandtheLapdog2 -> TheAssandtheLapdog
TheAssintheLionSkin -> TheAsstheFoxandtheLion
TheBellyandtheMembers2 -> TheBellyandtheMembers
TheBuffoonandtheCountryman -> TheBuffoonandtheCountryman2
TheCrowandthePitcher2 -> crowpitc2.ram
TheDogintheManger2 -> TheHeiferandtheOx
TheDogandtheShadow2 -> TheDogandtheShadow
TheEagleandtheArrow2 -> TheEagleandtheArrow
TheFoxandtheCrow2 -> TheFoxandtheCrow
TheFoxandtheGoat2 -> TheFoxandtheGoat
TheFoxandtheGrapes2 -> TheFoxandtheGrapes
TheFoxandtheLion -> TheFoxandtheLion2
TheFoxandtheMask -> foxmask2.ram
haretort2.ram -> TheHareandtheTortoise
harefrog2.ram -> TheHaresandtheFrogs
TheHorseandtheAss2 -> TheHorseandtheAss
TheLionandtheMouse2 -> lionmouse
TheLioninLove2 -> TheLioninLove
TheManandtheSatyr2 -> TheManandtheSatyr
MercuryandtheWoodman -> MercuryandtheWorkmen
milkpail2.ram -> milkmaidjug.jpg
TheOldManandDeath2 -> TheFatherandHisSons
TheOldWomanandtheWine